<a href="https://colab.research.google.com/github/nic-fp/Learning-Machine-Learning/blob/master/TitanicDataSetFocusOnProcessingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
from __future__ import absolute_import, division, print_function, unicode_literals
import functools
import numpy as np
import tensorflow as tf
import pandas as pd

#importing data
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

LABEL_COLUMN = "survived"
LABELS = [0,1]
#creates a csv data set from link
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size = 5,
      label_name= LABEL_COLUMN,
      na_value= "?",
      num_epochs =1, 
      ignore_errors = True,
      **kwargs)
  return dataset

unpacked_test_data= get_dataset(test_file_path)
unpacked_train_data = get_dataset(train_file_path)



COLUMN_NAMES =['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)
                            #Might need this instead


temp_dataset = get_dataset(train_file_path, column_names= COLUMN_NAMES)

#class to put all numeric data into a single column
class PackedNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_freatures = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_freatures]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

NUMERIC_FEATURES= ['age','n_siblings_spouses','parch', 'fare']
packed_train_data = unpacked_train_data.map(PackedNumericFeatures(NUMERIC_FEATURES))
packed_test_data = unpacked_test_data.map(PackedNumericFeatures(NUMERIC_FEATURES))

#normalize data
data_description = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
MEAN = np.array(data_description.T['mean'])
STD = np.array(data_description.T['std'])

def normalizer(data,mean,std):
  return (data-mean)/std

normalized = functools.partial(normalizer, mean= MEAN, std=STD)
numeric_column = tf.feature_column.numeric_column('numeric',normalizer_fn= normalized,shape=[len(NUMERIC_FEATURES)])
numeric_columns =  [numeric_column]

numeric_layer = tf.keras.layers.DenseFeatures(feature_columns=numeric_columns)


#Catagorical data

CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
    }

categorical_list = []

for feature, term in CATEGORIES.items():
  categorical_column = tf.feature_column.categorical_column_with_vocabulary_list(key=feature,vocabulary_list=term)
  categorical_list.append(tf.feature_column.indicator_column(categorical_column))

categorical_layer = tf.keras.layers.DenseFeatures(feature_columns=categorical_list)

combined_layer = tf.keras.layers.DenseFeatures(feature_columns= numeric_columns+categorical_list)

model = tf.keras.Sequential([
                             combined_layer, 
                             tf.keras.layers.Dense(128, activation = 'relu'), 
                             tf.keras.layers.Dense(128,activation = 'relu'),
                             tf.keras.layers.Dense(1,activation= 'sigmoid')
                             ])
model.compile(loss='binary_crossentropy', optimizer ='adam', metrics= ['accuracy'])

training_data = packed_train_data.shuffle(500)
testing_data = packed_test_data



model.fit(training_data,epochs= 30)

test_loss,accuracy = model.evaluate(testing_data)
"""print('\n\nTest Loss {}, Test Accuracy {}').format(test_loss,accuracy)

predictions = model.predict(testing_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))"""

Epoch 1/30
126/126 [==============================] - 2s 14ms/step - loss: 0.5009 - accuracy: 0.7799
Epoch 2/30
126/126 [==============================] - 0s 3ms/step - loss: 0.4179 - accuracy: 0.8150
Epoch 3/30
126/126 [==============================] - 0s 3ms/step - loss: 0.4003 - accuracy: 0.8262
Epoch 4/30
126/126 [==============================] - 1s 4ms/step - loss: 0.3849 - accuracy: 0.8325
Epoch 5/30
126/126 [==============================] - 1s 4ms/step - loss: 0.3713 - accuracy: 0.8373
Epoch 6/30
126/126 [==============================] - 1s 5ms/step - loss: 0.3716 - accuracy: 0.8421
Epoch 7/30
126/126 [==============================] - 0s 4ms/step - loss: 0.3681 - accuracy: 0.8389
Epoch 8/30
126/126 [==============================] - 0s 3ms/step - loss: 0.3563 - accuracy: 0.8341
Epoch 9/30
126/126 [==============================] - 0s 3ms/step - loss: 0.3531 - accuracy: 0.8517
Epoch 10/30
126/126 [==============================] - 0s 3ms/step - loss: 0.3605 - accuracy: 0.848